In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sys 
sys.path.append('/home1/efeghhi/ripple_memory/analysis_code/')
from load_data import *
from analyze_data import *
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import decimate, resample

from mne.time_frequency import tfr_array_morlet
from scipy.signal import hilbert
from load_data_numpy import load_data_np

from pactools import Comodulogram, REFERENCES
from pactools.comodulogram import read_comodulogram
from pactools.bandpass_filter import multiple_band_pass
from pactools import simulate_pac
from theta_gamma_PAC import compute_entropy


/home1/efeghhi/.conda/envs/env3/lib/python3.7/site-packages/ptsa/data/timeseries.py:526: FutureWarning: xarray subclass TimeSeriesX should explicitly define __slots__
  class TimeSeriesX(TimeSeries):
/home1/efeghhi/.conda/envs/env3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/home1/efeghhi/.conda/envs/env3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home1/efeghhi/.conda/envs/env3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home1/efeghhi/.conda/envs/env3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.u

In [2]:
run_mode = 1

def remove_session_string(subject_session_elec_string):
    
    subj_sess = subject_session_elec_string.split('_')[0]
    subj = subj_sess.split('-')[0]
    
    elec = subject_session_elec_string.split('_')[1]
    
    subj_elec = f"{subj}_{elec}"
    
    return subj_elec

if run_mode == 1:
    encoding_mode = 1
    saveName = 'encoding_'
    
if run_mode == 2:
    encoding_mode = 0
    saveName = 'recall_'
    
dd_trials = load_data_np(encoding_mode)

if encoding_mode:
    # relative to word onset 
    start_roi = 300
    end_roi = 1300
    
    # each trial consists of 5 sec of raw data, which starts
    # 1.7 before word onset and ends 3.3 sec after word onset 
    # the data is sampled at 500 Hz
    start_time = -1700
    end_time = 3300

else:
    
    start_roi = -1100
    end_roi = -100
    
    # for recall, 
    # each trial consists of 6 sec of data, centered around word recall 
    start_time = -3000
    end_time = 3000
    
sr_factor = 2

# convert to indices based on start time and sampling rate factor
start_idx = int((start_roi - start_time)/sr_factor)
end_idx = int((end_roi-start_time)/sr_factor)

raw_data = dd_trials['raw']
clust = dd_trials['clust_int']

nonclust_idxs = np.argwhere(clust==0)
clust_idxs = np.argwhere(clust==1)
subj_elec_labels = np.array([remove_session_string(x) for x in dd_trials['elec_labels']])
subj_elec_clust_idxs = subj_elec_labels[clust_idxs].squeeze()
subj_elec_nonclust_idxs = subj_elec_labels[nonclust_idxs].squeeze()
raw_data_clust = raw_data[clust_idxs].squeeze()
raw_data_not_clust = raw_data[nonclust_idxs].squeeze()

/home1/efeghhi/.conda/envs/env3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Loading data
Number of trials in ca1: 55356
order: C


In [ ]:
savePath = '/home1/efeghhi/ripple_memory/analysis_code/pac_analyses/saved_results/comodulogram/'
for subj_elec in np.unique(subj_elec_clust_idxs):
    
    subj_elec_idxs = np.argwhere(subj_elec_clust_idxs==subj_elec).squeeze()
    
    if subj_elec_idxs.shape[0] < 10:
        continue
        
    raw_data_clust_elec_subj = raw_data_clust[subj_elec_idxs].squeeze()
    
    low_fq_range = [[2,4],[5,10]]
    high_fq_range = [[30,70], [80,120], [130,160]]
    fs = 500
    low_fq_width = 1.0  # Hz
    progress_bar = True
    method = 'tort'
    
    mask = np.ones_like(raw_data_clust_elec_subj)
    mask[:, start_idx:end_idx] = 0

    estimator = Comodulogram(fs=fs, low_fq_range=low_fq_range, high_fq_range=high_fq_range, 
                             low_fq_width=low_fq_width, method=method,
                             progress_bar=progress_bar, n_surrogates=300, 
                             minimum_shift=0.1)
    
    estimator.fit(raw_data_clust_elec_subj, mask=mask)
    fig, ax = plt.subplots(1,2, figsize=(16,6))
    estimator.plot(axs=[ax[0]])
    sns.heatmap(estimator.comod_z_score_.T, cmap='viridis', yticklabels=high_fq_range[::-1], xticklabels=low_fq_range)
    plt.savefig(f'{savePath}{subj_elec}', dpi=300, bbox_inches='tight')
    plt.show()
    np.save(f'{savePath}{subj_elec}_comod_zscore', estimator.comod_z_score_.T)
    np.save(f'{savePath}{subj_elec}_comod', estimator.comod_.T)
  

In [ ]:
savePath = '/home1/efeghhi/ripple_memory/analysis_code/pac_analyses/saved_results/comodulogram/not_clust/'
for subj_elec in np.unique(subj_elec_not_clust_idxs):
    
    subj_elec_idxs = np.argwhere(subj_elec_not_clust_idxs==subj_elec).squeeze()
    
    if subj_elec_idxs.shape[0] < 10:
        continue
        
    raw_data_clust_elec_subj = raw_data_clust[subj_elec_idxs].squeeze()
    
    low_fq_range = [[2,4],[5,10]]
    high_fq_range = [[30,70], [80,120], [130,160]]
    fs = 500
    low_fq_width = 1.0  # Hz
    progress_bar = True
    method = 'tort'
    
    mask = np.ones_like(raw_data_clust_elec_subj)
    mask[:, start_idx:end_idx] = 0

    estimator = Comodulogram(fs=fs, low_fq_range=low_fq_range, high_fq_range=high_fq_range, 
                             low_fq_width=low_fq_width, method=method,
                             progress_bar=progress_bar, n_surrogates=300, 
                             minimum_shift=0.1)
    
    estimator.fit(raw_data_clust_elec_subj, mask=mask)
    fig, ax = plt.subplots(1,2, figsize=(16,6))
    estimator.plot(axs=[ax[0]])
    sns.heatmap(estimator.comod_z_score_.T, cmap='viridis', yticklabels=high_fq_range[::-1], xticklabels=low_fq_range)
    plt.savefig(f'{savePath}{subj_elec}', dpi=300, bbox_inches='tight')
    plt.show()
    np.save(f'{savePath}{subj_elec}_comod_zscore', estimator.comod_z_score_.T)
    np.save(f'{savePath}{subj_elec}_comod', estimator.comod_.T)